In [1]:
# [30 sampling] cek untuk overlap:
# substring, superstring, intersection
# (ada kata yang sama, namun tidak semua kata)
# untuk semua RA, RR, WA, WR.

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from collections import OrderedDict
import pandas as pd
import re
import ast
from tqdm import tqdm
import string
import collections
import numpy as np

from io import StringIO
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [3]:
prefix = "https://huggingface.co/muhammadravi251001/fine-tuned-FilteringNLI-"
suffix = "/blob/main/results/evaluation/metric_result.txt"

suffix_df = "/raw/main/results/output/output_df.csv"
suffix_squadid_df = "/resolve/main/results/output/output_df.csv"

In [4]:
def take_dataframe(url): 
    
    response = requests.get(url)
    
    if response.status_code == 200:
        csv_data = StringIO(response.text)
        df = pd.read_csv(csv_data, index_col=0)
        df = df[['Context', 'Question', 'Prediction Answer', 'Rec. Pred Answer', 'Gold Answer', 'Properties']]
        df.fillna('', inplace=True)
    
    else:
        print("Failed to download CSV")
    
    return df

In [5]:
def remove_punctuation(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

def is_substring(text1, text2):
    
    text1 = remove_punctuation(text1)
    text2 = remove_punctuation(text2)
    
    if text1 == text2 or text1 == "" or text2 == "": 
        return False
    
    return text1 in text2

def is_superstring(text1, text2):
    
    text1 = remove_punctuation(text1)
    text2 = remove_punctuation(text2)
    
    if text1 == text2 or text1 == "" or text2 == "":
        return False
    
    return text2 in text1

def is_intersection(text1, text2):
    
    text1 = remove_punctuation(text1)
    text2 = remove_punctuation(text2)
    
    if text1 == text2 or text1 == "" or text2 == "":
        return False
    
    set1 = set(text1.split())
    set2 = set(text2.split())
    
    return bool(set1.intersection(set2)) and \
            bool(set1.difference(set2)) and \
            bool(set2.difference(set1))

In [6]:
def read_clearly(df):
    
    for i in range(len(df)):
        
        print("Iterasi:", i)
        print()
        
        print("Context:")
        print(df["Context"][i])
        print()

        print("Question:")
        print(df["Question"][i])
        print()

        print("NLI Validated Answer:")
        print(df["NLI Validated Answer"][i])
        print()

        print("QA Generated Answer:")
        print(df["QA Generated Answer"][i])
        print()

        print("Gold Answer:")
        print(df["Gold Answer"][i])
        print()
        
        print("=" * 30)

In [20]:
data = "idkmrc"
msc = f"indonli_mnli_{data}-nli"
tq = "1"
ts = "4"
msi = "1"
var = "2"
th = "0.75"

url = f"{prefix}{msc}-{data}-TQ{tq}-TS{ts}-MS{msi}-VA{var}-TH{th}{suffix_df}"

df = pd.DataFrame(take_dataframe(url))

df = df.drop(columns=["Properties"])
df['Rec. Pred Answer'] = df['Rec. Pred Answer'].apply(lambda x: ast.literal_eval(x)[0] if isinstance(x, str) else None)
df = df.rename(columns={'Prediction Answer': 'NLI Validated Answer', 
                        'Rec. Pred Answer': 'QA Generated Answer'})
df

,Context,Question,NLI Validated Answer,QA Generated Answer,Gold Answer
0,Ada beberapa inovasi dari GAN yang sedikit nga...,Kapan Komputer mikro mulai dikembangkan ?,1959,1959,1959
1,Ada beberapa inovasi dari GAN yang sedikit nga...,Kapan komputer mikro mulai ditinggalkan?,,,
2,Ada beberapa inovasi dari GAN yang sedikit nga...,Kapan Komputer mikro mulai dikembangkan Amerik...,,,
3,Ada beberapa inovasi dari GAN yang sedikit nga...,Kapan Komputer makro mulai dikembangkan?,,,
4,"Patronim, atau patronimik, adalah sebuah kompo...",Apakah pengertian matronimik?,komponen dari sebuah nama pribadi yang berdasa...,komponen dari sebuah nama pribadi yang berdasa...,Komomene sebuah nama yang berdasarkan pada nam...
...,...,...,...,...,...
839,Umur dewasa dari drummer ganda adalah sekitar ...,Bagaimana cara jangkrik betina hidup?,,,
840,Kota ini terletak di sebuah lembah sungai yang...,"Menurut Biro Sensus Amerika Serikat, berapa lu...",131.3 mil persegi,131.3 mil persegi,"Menurut Biro Sensus Amerika Serikat, kota ini ..."
841,Kota ini terletak di sebuah lembah sungai yang...,"Menurut Biro Sensus Amerika Serikat, berapa vo...",,,
842,"Tanpa beasiswa, Ogilvy tidak bisa kuliah di Fe...",Apa alasan Ogilvy tidak bisa kuliah di Fettes ...,bisnis ayahnya terkena dampak depresi pertenga...,bisnis ayahnya terkena dampak depresi pertenga...,Ogilvy tidak bisa kuliah di Fettes atau Oxford...


In [8]:
df['Is Substring'] = df.apply(lambda row: is_substring(row['NLI Validated Answer'], row['Gold Answer']), axis=1)
df['Is Superstring'] = df.apply(lambda row: is_superstring(row['NLI Validated Answer'], row['Gold Answer']), axis=1)
df['Is Intersection'] = df.apply(lambda row: is_intersection(row['NLI Validated Answer'], row['Gold Answer']), axis=1)

#sample_is_substring = df[df['Is Substring']].sample(10, random_state=2023)
#sample_is_superstring = df[df['Is Superstring']].sample(10, random_state=2023)
#sample_is_intersection = df[df['Is Intersection']].sample(10, random_state=2023)

#dropped_column = ['Is Substring', 'Is Superstring', 'Is Intersection']
#sample_is_substring = sample_is_substring.drop(columns=dropped_column).reset_index(drop=True)
#sample_is_superstring = sample_is_superstring.drop(columns=dropped_column).reset_index(drop=True)
#sample_is_intersection = sample_is_intersection.drop(columns=dropped_column).reset_index(drop=True)

In [9]:
import pandas as pd

def take_accepted_and_rejected_qa_with_nli(df):

    right_pred_model_qa_and_accepted_by_nli_df = pd.DataFrame()
    right_pred_model_qa_and_rejected_by_nli_df = pd.DataFrame()
    wrong_pred_model_qa_and_accepted_by_nli_df = pd.DataFrame()
    wrong_pred_model_qa_and_rejected_by_nli_df = pd.DataFrame()

    for i in range(len(df)):
        
        gold_answer = df['Gold Answer'][i]
        qa_generated_answer = df['QA Generated Answer'][i]
        nli_validated_answer = df['NLI Validated Answer'][i]

        # Jawaban dari model qa yg benar & di-accept sama model NLI
        right_pred_model_qa_and_accepted_by_nli = (gold_answer == qa_generated_answer) and \
            (qa_generated_answer == nli_validated_answer)

        # Jawaban dari model qa yg benar & di-reject sama model NLI
        right_pred_model_qa_and_rejected_by_nli = (gold_answer == qa_generated_answer) and \
            (qa_generated_answer != nli_validated_answer)

        # Jawaban dari model qa yg salah & di-accept sama model NLI
        wrong_pred_model_qa_and_accepted_by_nli = (gold_answer != qa_generated_answer) and \
            (qa_generated_answer == nli_validated_answer)

        # Jawaban dari model qa yg salah & di-reject sama model NLI
        wrong_pred_model_qa_and_rejected_by_nli = (gold_answer != qa_generated_answer) and \
            (qa_generated_answer != nli_validated_answer)

        if right_pred_model_qa_and_accepted_by_nli:
            right_pred_model_qa_and_accepted_by_nli_df = right_pred_model_qa_and_accepted_by_nli_df.append(df.iloc[i])
        elif right_pred_model_qa_and_rejected_by_nli:
            right_pred_model_qa_and_rejected_by_nli_df = right_pred_model_qa_and_rejected_by_nli_df.append(df.iloc[i])
        elif wrong_pred_model_qa_and_accepted_by_nli:
            wrong_pred_model_qa_and_accepted_by_nli_df = wrong_pred_model_qa_and_accepted_by_nli_df.append(df.iloc[i])
        elif wrong_pred_model_qa_and_rejected_by_nli:
            wrong_pred_model_qa_and_rejected_by_nli_df = wrong_pred_model_qa_and_rejected_by_nli_df.append(df.iloc[i])

    right_pred_model_qa_and_accepted_by_nli_df = right_pred_model_qa_and_accepted_by_nli_df.reset_index(drop=True)
    right_pred_model_qa_and_rejected_by_nli_df = right_pred_model_qa_and_rejected_by_nli_df.reset_index(drop=True)
    wrong_pred_model_qa_and_accepted_by_nli_df = wrong_pred_model_qa_and_accepted_by_nli_df.reset_index(drop=True)
    wrong_pred_model_qa_and_rejected_by_nli_df = wrong_pred_model_qa_and_rejected_by_nli_df.reset_index(drop=True)
    
    return right_pred_model_qa_and_accepted_by_nli_df, right_pred_model_qa_and_rejected_by_nli_df, \
           wrong_pred_model_qa_and_accepted_by_nli_df, wrong_pred_model_qa_and_rejected_by_nli_df

In [10]:
def count_overlapping(df):
    print("NLI validated answer as Substring of gold truth:", int(df['Is Substring'].sum()))
    print("NLI validated answer as Superstring of gold truth:", int(df['Is Superstring'].sum()))
    print("NLI validated answer as Intersection of gold truth:", int(df['Is Intersection'].sum()))

In [11]:
ra, rr, wa, wr = take_accepted_and_rejected_qa_with_nli(df)

In [12]:
print("IDK MRC")
print()

dataframes = {'RA': ra, 'RR': rr, 'WA': wa, 'WR': wr}

for label, df in dataframes.items():
    print(label)
    count_overlapping(df)
    print()

IDK MRC

RA
NLI validated answer as Substring of gold truth: 0
NLI validated answer as Superstring of gold truth: 0
NLI validated answer as Intersection of gold truth: 0

RR
NLI validated answer as Substring of gold truth: 0
NLI validated answer as Superstring of gold truth: 0
NLI validated answer as Intersection of gold truth: 0

WA
NLI validated answer as Substring of gold truth: 42
NLI validated answer as Superstring of gold truth: 42
NLI validated answer as Intersection of gold truth: 11

WR
NLI validated answer as Substring of gold truth: 0
NLI validated answer as Superstring of gold truth: 0
NLI validated answer as Intersection of gold truth: 0



In [13]:
data = "tydiqaid"
msc = f"indonli_mnli_{data}-nli"
tq = "1"
ts = "4"
msi = "2"
var = "3"
th = "0.5"

url = f"{prefix}{msc}-{data}-TQ{tq}-TS{ts}-MS{msi}-VA{var}-TH{th}{suffix_df}"

df = pd.DataFrame(take_dataframe(url))

df = df.drop(columns=["Properties"])
df['Rec. Pred Answer'] = df['Rec. Pred Answer'].apply(lambda x: ast.literal_eval(x)[0] if isinstance(x, str) else None)
df = df.rename(columns={'Prediction Answer': 'NLI Validated Answer', 
                        'Rec. Pred Answer': 'QA Generated Answer'})

df['Is Substring'] = df.apply(lambda row: is_substring(row['NLI Validated Answer'], row['Gold Answer']), axis=1)
df['Is Superstring'] = df.apply(lambda row: is_superstring(row['NLI Validated Answer'], row['Gold Answer']), axis=1)
df['Is Intersection'] = df.apply(lambda row: is_intersection(row['NLI Validated Answer'], row['Gold Answer']), axis=1)

ra, rr, wa, wr = take_accepted_and_rejected_qa_with_nli(df)

In [14]:
print("TYDIQA ID")
print()

dataframes = {'RA': ra, 'RR': rr, 'WA': wa, 'WR': wr}

for label, df in dataframes.items():
    print(label)
    count_overlapping(df)
    print()

TYDIQA ID

RA
NLI validated answer as Substring of gold truth: 0
NLI validated answer as Superstring of gold truth: 0
NLI validated answer as Intersection of gold truth: 0

RR
NLI validated answer as Substring of gold truth: 0
NLI validated answer as Superstring of gold truth: 0
NLI validated answer as Intersection of gold truth: 0

WA
NLI validated answer as Substring of gold truth: 53
NLI validated answer as Superstring of gold truth: 110
NLI validated answer as Intersection of gold truth: 45

WR
NLI validated answer as Substring of gold truth: 1
NLI validated answer as Superstring of gold truth: 17
NLI validated answer as Intersection of gold truth: 3



In [17]:
data = "squadid"
msc = f"indonli_mnli_{data}-nli"
tq = "2"
ts = "4"
msi = "2"
var = "3"
th = "0.5"

url = f"{prefix}{msc}-{data}-TQ{tq}-TS{ts}-MS{msi}-VA{var}-TH{th}{suffix_squadid_df}"

df = pd.DataFrame(take_dataframe(url))

df = df.drop(columns=["Properties"])
df['Rec. Pred Answer'] = df['Rec. Pred Answer'].apply(lambda x: ast.literal_eval(x)[0] if isinstance(x, str) else None)
df = df.rename(columns={'Prediction Answer': 'NLI Validated Answer', 
                        'Rec. Pred Answer': 'QA Generated Answer'})

df['Is Substring'] = df.apply(lambda row: is_substring(row['NLI Validated Answer'], row['Gold Answer']), axis=1)
df['Is Superstring'] = df.apply(lambda row: is_superstring(row['NLI Validated Answer'], row['Gold Answer']), axis=1)
df['Is Intersection'] = df.apply(lambda row: is_intersection(row['NLI Validated Answer'], row['Gold Answer']), axis=1)

ra, rr, wa, wr = take_accepted_and_rejected_qa_with_nli(df)

In [18]:
print("SQUAD ID")
print()

dataframes = {'RA': ra, 'RR': rr, 'WA': wa, 'WR': wr}

for label, df in dataframes.items():
    print(label)
    count_overlapping(df)
    print()

SQUAD ID

RA
NLI validated answer as Substring of gold truth: 0
NLI validated answer as Superstring of gold truth: 0
NLI validated answer as Intersection of gold truth: 0

RR
NLI validated answer as Substring of gold truth: 5
NLI validated answer as Superstring of gold truth: 3
NLI validated answer as Intersection of gold truth: 2

WA
NLI validated answer as Substring of gold truth: 1187
NLI validated answer as Superstring of gold truth: 1795
NLI validated answer as Intersection of gold truth: 571

WR
NLI validated answer as Substring of gold truth: 15
NLI validated answer as Superstring of gold truth: 46
NLI validated answer as Intersection of gold truth: 13

